In [14]:
from typing import List

In [2]:
import spacy

In [3]:
nlp = spacy.load("en_core_web_lg")

In [4]:
txt1 = "And they gave some of the most wonderful gifts to everyone."
doc1 = nlp(txt1)

In [8]:
for token in doc1:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.head, [child.text for child in token.children],
            token.shape_, token.is_alpha, token.is_stop)

And and CCONJ CC cc gave [] Xxx True True
they they PRON PRP nsubj gave [] xxxx True True
gave give VERB VBD ROOT gave ['And', 'they', 'some', 'to', '.'] xxxx True False
some some PRON DT dobj gave ['of'] xxxx True True
of of ADP IN prep some ['gifts'] xx True True
the the DET DT det gifts [] xxx True True
most most ADV RBS advmod wonderful [] xxxx True True
wonderful wonderful ADJ JJ amod gifts ['most'] xxxx True False
gifts gift NOUN NNS pobj of ['the', 'wonderful'] xxxx True False
to to ADP IN dative gave ['everyone'] xx True True
everyone everyone PRON NN pobj to [] xxxx True True
. . PUNCT . punct gave [] . False False


In [9]:
spacy.displacy.render(doc1, style="dep", jupyter=True)

In [10]:
DISTRIBUTIVE_FRAME = {
    "priority": 1,
    "slots": {
        0: {
            "variable_or_constant": "constant",
            "synt_type": "DET",
            "roles": {"": 1},
            "head": None,
            "requirements": [
                "both",
                "either",
                "neither",
                "every",
                "each",
                "all",
                "none",
                "every one",
                "everyone",
                "some",
            ],
            "form": "",
            "requirement_type": "required",
        },
        1: {
            "variable_or_constant": "constant",
            "synt_type": "ADP",
            "roles": {"prep": 1},
            "head": 0,
            "requirements": "of",
            "form": "",
            "requirement_type": "required",
        },
        2: {
            "variable_or_constant": "variable",
            "synt_type": "NP",
            "roles": {"pobj": 1},
            "head": 1,
            "requirements": "NP",
            "form": "",
            "requirement_type": "required",
        },
    },
    "dependent_processes": ["NPCreator"],
}

In [18]:
class FrameSlot:
    @classmethod
    def from_slot_dict(cls, slot_dict):
        frame_slot = FrameSlot()
        frame_slot.__dict__.update(**slot_dict)
        frame_slot.candidates = []
        frame_slot.bond = None

        return frame_slot

    def set_bond(self, candidate):
        self.bond = candidate

    def get_bond(self):
        return self.bond

    def set_form(self, form):
        self.form = form

    def add_candidate(self, candidate):
        self.candidates.append(candidate)

    def extend_candidates(self, candidate_list):
        self.candidates.extend(candidate_list)

    def remove_candidate(self, candidate):
        self.candidates.remove(candidate)

    def replace_candidates(self, candidate_list):
        self.candidates = candidate_list


class Frame:
    @classmethod
    def from_frame_dict(cls, frame_dict):
        frame = Frame()
        frame.frame_dict = frame_dict
        frame.slots = []
        # Candidate for deletion if not found useful until Nov 13
        # frame.required_slots_to_find = len([slot for slot in frame.slots if slot.optional == False])
        frame.all_required_slots_found = False

        # Candidate for deletion, if not found useful until Nov 4, 2021
        # frame.matcher = None
        frame.priority = frame_dict["priority"]
        slot_dict = frame_dict["slots"]
        for key in slot_dict.keys():
            frame.slots.append(FrameSlot.from_slot_dict(slot_dict[key]))
        # TODO: think how to process conditionally required slots
        frame.required_slots_to_find = len(
            [slot for slot in frame.slots if slot.requirement_type == "required"]
        )

        frame.num_dependent_processes = 0
#         if "dependent_processes" in frame_dict:
#             frame.dependent_processes = []
#             for dependent_process_name in frame_dict["dependent_processes"]:
#                 dependent_process_class = getattr(codelets, dependent_process_name)
#                 dependent_process = dependent_process_class()
#                 frame.dependent_processes.append(dependent_process)
#             frame.num_dependent_processes = len(frame.dependent_processes)

        return frame

    def set_bond(self, slot_id, candidate):
        self.slots[slot_id].set_bond(candidate)

    def get_bond(self, slot_id):
        return self.slots[slot_id].get_bond()

    def set_form(self, slot_id, form):
        self.slots[slot_id].set_form(form)

    def set_all_required_slots_found(self):
        self.all_required_slots_found = True

    def get_all_required_slots_found(self):
        return self.all_required_slots_found

    def reduce_required_slots_to_find(self, number_of_slots: int = 1):
        self.required_slots_to_find -= number_of_slots

    def get_all_bonded(self):
        return set().union(*[slot.bond for slot in self.slots if slot.bond is not None])

    def copy(self):
        return Frame.from_frame_dict(self.frame_dict)

    def __str__(self):
        frame_str = ""
        for key, value in self.__dict__.items():
            if isinstance(value, List):
                for item in value:
                    if isinstance(item, FrameSlot):
                        frame_str += str(item.__dict__) + ", \n"
                    elif isinstance(item, codelets.WorkSpaceModifier):
                        frame_str += f"({str(key)}, {str(item)})" + ", \n"
                    else:
                        frame_str += f"({str(key)}, {str(value)})"
        return frame_str


In [19]:
dist_frame = Frame.from_frame_dict(DISTRIBUTIVE_FRAME)

In [22]:
print(dist_frame)

{'variable_or_constant': 'constant', 'synt_type': 'DET', 'roles': {'': 1}, 'head': None, 'requirements': ['both', 'either', 'neither', 'every', 'each', 'all', 'none', 'every one', 'everyone', 'some'], 'form': '', 'requirement_type': 'required', 'candidates': [], 'bond': None}, 
{'variable_or_constant': 'constant', 'synt_type': 'ADP', 'roles': {'prep': 1}, 'head': 0, 'requirements': 'of', 'form': '', 'requirement_type': 'required', 'candidates': [], 'bond': None}, 
{'variable_or_constant': 'variable', 'synt_type': 'NP', 'roles': {'pobj': 1}, 'head': 1, 'requirements': 'NP', 'form': '', 'requirement_type': 'required', 'candidates': [], 'bond': None}, 



In [23]:
dist_frame.slots

In [25]:
dist_frame.slots[0].add_candidate([3])
dist_frame.slots[0].add_candidate([10])

In [26]:
dist_frame.slots[1].add_candidate([4])

In [27]:
dist_frame.slots[2].add_candidate([5, 6, 7, 8])

In [38]:
for slot in dist_frame.slots:
    print(slot.candidates)
    for candidate in slot.candidates:
        for canididate_ind in candidate:
            for child in doc1[candidate_ind].children:
                if child.i in 
            print([child.i for child in doc1[canididate_ind].children])

[[3], [10]]
2
[4]
9
[]
[[4]]
3
[8]
[[5, 6, 7, 8]]
8
[]
7
[]
8
[6]
4
[5, 7]


In [36]:
doc1[3].i

3